In [11]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [7]:
from utils.image_to_embedding import get_image_to_embedding
from utils.static_check import get_static_difference
from utils.optical_flow_check import get_optical_flow
from diffusers import AutoencoderKL
import torch

import cv2
import numpy as np
from PIL import Image
import os
from torchvision import transforms

def image_transform(image):
    transform = transforms.Compose([
        transforms.Resize((320, 240)),
        transforms.CenterCrop(240),
        transforms.ToTensor(),  
    ])
    return transform(image).unsqueeze(0)

def load_image(folder_path):
    # Sort the frames
    frames = sorted(os.listdir(folder_path))
    conv_frames = []
    
    # Convert the frames to tensor
    for frame in frames:
        image = Image.open(os.path.join(folder_path, frame)).convert('RGB')
        image = image_transform(image)
        conv_frames.append(image)
    return conv_frames

def get_model():
    model = AutoencoderKL.from_pretrained("stabilityai/sdxl-vae")
    return model

def get_inference(folder_path, model, device):
    frames = load_image(folder_path)

    # Getting static difference
    rgb_diff = get_static_difference(frames)

    # getting optical flow
    avg_velocity = get_optical_flow(frames)
    
    # Getting image context similarity
    frame_mse, frame_cos_sim = get_image_to_embedding(frames, model, device)

    return rgb_diff, avg_velocity, frame_mse, frame_cos_sim

In [8]:
rgb_diffs = []
gpb_diffs = []
avg_velocities = []
frame_mses = []
frame_cos_sims = []
labels = []

In [9]:
# Load from JV's Dataset

labeled_data = {
    "frames_output\\44ot5_2UaaU\\44ot5_2UaaU.4_7": 1,
    "frames_output\\44ot5_2UaaU\\44ot5_2UaaU.15_0": 0,
    "frames_output\\44ot5_2UaaU\\44ot5_2UaaU.4_1": 0,
    "frames_output\\1NRXqc74kQM\\1NRXqc74kQM.8_3": 1
}

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = get_model().to(device)

for i, path in enumerate(labeled_data.keys()):
    rgb_diff, avg_velocity, frame_mse, frame_cos_sim = get_inference(path, model, device)
    gpb_diffs.append(rgb_diff)
    avg_velocities.append(avg_velocity)
    frame_mses.append(frame_mse)
    frame_cos_sims.append(frame_cos_sim)
    labels.append(labeled_data[path])
    print(f'Enumerated {i+1} paths')

In [ ]:
import pandas as pd

dataset = pd.DataFrame({
    "rgb_diff": gpb_diffs,
    "avg_velocity": avg_velocities,
    "frame_mse": frame_mses,
    "frame_cos_sim": frame_cos_sims,
    "label": labels
})

In [ ]:
X = dataset.drop(columns=["label"])
y = dataset["label"]

In [ ]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creating an XGBoost classifier
model = xgb.XGBClassifier()

# Training the model
model.fit(X_train, y_train)

# Making predictions on the test set
y_pred = model.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
import pickle

# Save the trained model to a file
filename = 'xgboost_model.pth'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
# TO TEST LOADING THE SAVED MODEL

# Load the saved model from file
classifier_model = pickle.load(open(filename, 'rb'))

# Use the loaded model for predictions
y_pred_loaded = classifier_model.predict(X_test)

# Evaluate the loaded model
accuracy_loaded = accuracy_score(y_test, y_pred_loaded)
print("Accuracy of loaded model:", accuracy_loaded)